In [1]:
import pandas as pd
import numpy as np
import os

In [2]:

# --- 1. SETUP & LOAD ---
c_d = os.path.dirname(os.path.abspath('__file__')) 
data_path = '../TPP_Data/KaggleFootballData/'
processed_path = '../TPP_Data/TPP_ProcessedDatasets/'

print("Loading data...")
epl_arrivals = pd.read_csv(processed_path + 'epl_arrivals_with_country.csv')
player_history = pd.read_csv(processed_path + 'final_player_stats.csv')
games = pd.read_csv(data_path + 'games.csv')

# Conversions
epl_arrivals['transfer_date'] = pd.to_datetime(epl_arrivals['transfer_date'])
player_history['first_app'] = pd.to_datetime(player_history['first_app'])
player_history['last_app'] = pd.to_datetime(player_history['last_app'])
games['date'] = pd.to_datetime(games['date'])

# Create IDs
epl_arrivals['transfer_id'] = epl_arrivals['player_id'].astype(str) + "_" + epl_arrivals['transfer_date'].astype(str)


Loading data...


In [3]:


# --- 2. THE BIG MERGE (History < Transfer Date) ---
merged_df = epl_arrivals.merge(
    player_history, 
    on='player_id', 
    how='left', 
    suffixes=('_transfer', '_stint')
)

# Filter: Keep only PAST history
past_stats = merged_df[merged_df['last_app'] < merged_df['transfer_date']].copy()


In [4]:
# --- 3. THE "DEFENSIVE CONTEXT" ENGINE ---
print("Building Defensive Context (Goals Conceded & Clean Sheets)...")

# A. Build the Team Schedule (Now with OPPONENT GOALS)
# If Home: Team Goals = Home Goals, Opponent Goals = Away Goals
home_games = games[['home_club_id', 'date', 'home_club_goals', 'away_club_goals']].rename(
    columns={
        'home_club_id': 'club_id', 
        'home_club_goals': 'team_goals',
        'away_club_goals': 'opponent_goals'
    }
)

# If Away: Team Goals = Away Goals, Opponent Goals = Home Goals
away_games = games[['away_club_id', 'date', 'away_club_goals', 'home_club_goals']].rename(
    columns={
        'away_club_id': 'club_id', 
        'away_club_goals': 'team_goals',
        'home_club_goals': 'opponent_goals'
    }
)

team_schedule = pd.concat([home_games, away_games]).drop_duplicates()
team_schedule = team_schedule.sort_values(['club_id', 'date'])

# B. Calculate "Clean Sheet" for every game
# (1 if opponent scored 0, else 0)
team_schedule['clean_sheet'] = (team_schedule['opponent_goals'] == 0).astype(int)

# C. Inequality Join (Same as before)
unique_stints = past_stats[['player_id', 'player_club_id', 'first_app', 'last_app']].drop_duplicates()

team_schedule['date_num'] = team_schedule['date'].astype(np.int64)
unique_stints['start_num'] = unique_stints['first_app'].astype(np.int64)
unique_stints['end_num'] = unique_stints['last_app'].astype(np.int64)

stint_games = pd.merge(
    unique_stints, 
    team_schedule, 
    left_on='player_club_id', 
    right_on='club_id', 
    how='inner'
)

# Filter for date range
mask = (stint_games['date_num'] >= stint_games['start_num']) & \
       (stint_games['date_num'] <= stint_games['end_num'])
valid_games = stint_games[mask]

# D. Aggregate Defensive Stats per Stint
stint_context = valid_games.groupby(['player_id', 'player_club_id', 'first_app']).agg({
    'date': 'count',           # Games Played
    'team_goals': 'sum',       # For Dominance
    'opponent_goals': 'sum',   # NEW: Total Goals Conceded
    'clean_sheet': 'sum'       # NEW: Total Clean Sheets
}).reset_index().rename(columns={
    'date': 'actual_team_games', 
    'team_goals': 'actual_team_goals',
    'opponent_goals': 'actual_goals_conceded',
    'clean_sheet': 'actual_clean_sheets'
})

# E. Merge back
past_stats_with_context = past_stats.merge(
    stint_context, 
    on=['player_id', 'player_club_id', 'first_app'], 
    how='left'
).fillna(0)

past_stats_with_context['possible_minutes'] = past_stats_with_context['actual_team_games'] * 90


Building Defensive Context (Goals Conceded & Clean Sheets)...


1. Career Dominance (Goal Share)This measures a player's importance to their team's offense, independent of the league's overall scoring rate.
$$CareerDominance = \frac{\sum \text{Player Goals}}{\sum \text{Team Goals}}$$

Context: The denominator ($\sum \text{Team Goals}$) is strictly the total goals the team scored while the player was available (between their first and last appearance).Interpretation: A value of 0.35 means the player scored 35% of their team's goals.

2. Trajectory Score (Growth Rate)This measures whether a player is improving or declining at the time of transfer.

$$Trajectory = \frac{\text{Recent Goals per 90}}{\text{Career Goals per 90}}$$

Interpretation:$> 1.0$: Rising Star (Performance is accelerating).$< 1.0$: Declining (Performance is worse than their historical average).$\approx 1.0$: Consistent (Performance matches historical average).

Note: We also hard code the UEFA "league weights." These give a score that corresponds to the strength of the league. 

In [5]:


# --- 4. FEATURE ENGINEERING (Trajectory, Dominance, League) ---
print("Calculating Features...")

# A. Define League Weights (Same as before)
league_weights = {
    'Premier League': 1.0, 'LaLiga': 1.0, 'Bundesliga': 0.95, 'Serie A': 0.95,
    'Ligue 1': 0.85, 'Liga Portugal': 0.65, 'Eredivisie': 0.60,
    'Championship': 0.55, 'Süper Lig': 0.45, 'Jupiler Pro League': 0.45,
    'Série A': 0.40
}

# B. Create "Recent" Subset (Last 365 Days)
past_stats_with_context['days_before_transfer'] = (
    past_stats_with_context['transfer_date'] - past_stats_with_context['last_app']
).dt.days

recent_mask = past_stats_with_context['days_before_transfer'] <= 365
recent_stats = past_stats_with_context[recent_mask]

# C. Aggregation Function (UPDATED TO INCLUDE DEFENSE)
def aggregate_features(df, prefix):
    agg = df.groupby('transfer_id').agg({
        # Offense
        'goals': 'sum',
        'assists': 'sum',
        
        # Duration / Magnitude
        'minutes_played': 'sum',
        'possible_minutes': 'sum',
        'actual_team_games': 'sum',
        
        # Defense (NEWLY ADDED HERE)
        'actual_team_goals': 'sum',        # For Dominance
        'actual_goals_conceded': 'sum',    # For Conceded p90
        'actual_clean_sheets': 'sum',      # For Clean Sheet %
        'yellow_cards': 'sum',
        'red_cards': 'sum',

        # Metadata
        'origin_league_name': 'first',
        'player_name_transfer': 'first', 
        'age': 'first',
        'market_value_in_eur': 'first',
        'transfer_fee': 'first'
    }).reset_index()
    
    # Prefix columns (except ID and Metadata)
    meta_cols = ['transfer_id', 'origin_league_name', 'player_name_transfer', 'age', 'market_value_in_eur', 'transfer_fee']
    new_cols = {c: f"{prefix}_{c}" for c in agg.columns if c not in meta_cols}
    agg = agg.rename(columns=new_cols)
    return agg

# Aggregate Career and Recent
X_career = aggregate_features(past_stats_with_context, 'career')
X_recent = aggregate_features(recent_stats, 'recent')

# Join them
X_final = X_career.merge(
    X_recent.drop(columns=['origin_league_name', 'player_name_transfer', 'age', 'market_value_in_eur', 'transfer_fee']), 
    on='transfer_id', 
    how='left'
).fillna(0)

Calculating Features...


In [7]:
# --- 5. CALCULATE FINAL METRICS (UPDATED) ---

# Metric 1: Efficiency (Goals per 90)
X_final['career_goals_p90'] = (X_final['career_goals'] / X_final['career_minutes_played'].replace(0, np.nan)) * 90
X_final['recent_goals_p90'] = (X_final['recent_goals'] / X_final['recent_minutes_played'].replace(0, np.nan)) * 90

# Metric 2: Availability (Minutes / Possible)
X_final['career_availability'] = X_final['career_minutes_played'] / X_final['career_possible_minutes'].replace(0, np.nan)

# Metric 3: DOMINANCE (Player Goals / Team Goals)
X_final['career_dominance'] = X_final['career_goals'] / X_final['career_actual_team_goals'].replace(0, np.nan)
X_final['recent_dominance'] = X_final['recent_goals'] / X_final['recent_actual_team_goals'].replace(0, np.nan)

# Metric 4: TRAJECTORY (Recent p90 / Career p90)
X_final['trajectory_goals'] = X_final['recent_goals_p90'] / (X_final['career_goals_p90'] + 0.01)

# Metric 5: LEAGUE ADJUSTMENT
X_final['league_factor'] = X_final['origin_league_name'].map(league_weights).fillna(0.3)
X_final['adj_goals_p90'] = X_final['recent_goals_p90'] * X_final['league_factor']

# Metric 6: DEFENSIVE SOLIDITY (NEW)
# Clean Sheet % (Clean Sheets / Total Games Team Played)
X_final['clean_sheet_pct'] = X_final['career_actual_clean_sheets'] / X_final['career_actual_team_games'].replace(0, np.nan)

# Goals Conceded Per 90 (Goals Team Allowed / Time Player Could Have Played)
# Note: Using 'actual_team_games' is safer than minutes for the denominator here to act as "Per Game"
X_final['conceded_per_game'] = X_final['career_actual_goals_conceded'] / X_final['career_actual_team_games'].replace(0, np.nan)

# Metric 7: DISCIPLINE
X_final['card_points_p90'] = ((X_final['career_yellow_cards'] + X_final['career_red_cards']*3) / X_final['career_minutes_played'].replace(0, np.nan)) * 90

# Cleanup
X_final = X_final.fillna(0)

print("Final Feature Set Created.")
display(X_final[['player_name_transfer', 'origin_league_name', 'career_dominance', 'clean_sheet_pct', 'conceded_per_game']].head())

# Save
#if not os.path.exists(processed_path): os.makedirs(processed_path)
# X_final.to_csv(processed_path + 'X_features_master.csv', index=False)

Final Feature Set Created.


,player_name_transfer,origin_league_name,career_dominance,clean_sheet_pct,conceded_per_game
0,Mathis Amougou,ligue-1,0.000000,0.157895,2.052632
1,Joel Robles,laliga,0.000000,0.315789,1.526316
2,Joel Robles,laliga,0.000000,0.304183,1.425856
3,Pedro Obiang,serie-a,0.025806,0.256198,1.363636
4,Yankuba Minteh,superligaen,0.105263,0.214286,1.571429
